In [2]:
import pickle
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
import pandas as pd


df_pivot = pd.read_csv('pivot_table (1).csv', index_col=0)
df_melt = df_pivot.stack().reset_index().rename(columns={'level_1': 'Movie_Id', 0: 'Rating'})

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_melt[['Cust_Id', 'Movie_Id', 'Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)


n_factors = 10
n_epochs = 30
lr_all = 0.005
reg_all = 0.02

algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)
algo.fit(trainset)


model_path = 'best_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(algo, f)

print(f"Model sačuvan na {model_path}")


with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

print("Model uspešno učitan!")


df_new_test = pd.read_csv('test_data.csv')  


new_testset = [(row['Cust_Id'], row['Movie_Id'], row['Rating']) for _, row in df_new_test.iterrows()]


predictions = loaded_model.test(new_testset)


rmse = accuracy.rmse(predictions, verbose=True)
mae = accuracy.mae(predictions, verbose=True)

print(f'RMSE na novom test skupu: {rmse}')
print(f'MAE na novom test skupu: {mae}')


Model sačuvan na best_model.pkl
Model uspešno učitan!
RMSE: 0.9938
MAE:  0.7985
RMSE na novom test skupu: 0.9938370587068346
MAE na novom test skupu: 0.7985280018832762
